In [1]:
#Se quiere importar un archivo pickle con un diccionario y transformarlo en un dataframe de pandas

import pandas as pd
import pickle
import re
#Se importa el archivo pickle
with open('scrap_info_alq.pkl', 'rb') as f:
    diccionario = pickle.load(f)


with open('scrap_info.pkl', 'rb') as f:
    diccionario2 = pickle.load(f)

with open('scrap_info_pisos_com.pkl', 'rb') as f:
    diccionario3 = pickle.load(f)
#Se transforma el diccionario en un dataframe
df = pd.DataFrame(diccionario)
df2 = pd.DataFrame(diccionario2)
df3 = pd.DataFrame(diccionario3)
#Ahora se cambia las filas por las columnas
df = df.T
df2 = df2.T
df3 = df3.T
df.head()
df2.head()

#Ahora unimos los dos dataframes
df_def1 = pd.concat([df,df2])
df_def = pd.concat([df_def1,df3])
#Limpiamos la columna de precio quitando el simbolo de € y quitando el /mes

df_def['Precio'] = df_def['Precio'].str.replace('€','')
df_def['Precio'] = df_def['Precio'].str.replace('/mes','')

#transformamos el el precio a numerico	
df_def['Precio'] = df_def['Precio'].str.replace('.','')

#Eliminamos la fila que Precio que pone por consultar
df_def = df_def[df_def['Precio'] != 'precio a consultar']
df_def = df_def[df_def['Precio'] != 'A consultar ']
df_def = df_def[df_def["Precio"] != "nan"]
def extraer_primer_valor(cadena):
    # Usar una expresión regular para encontrar el primer número en la cadena
    primer_numero = re.search(r'\d+', cadena)
    # Devolver el primer número si existe
    return int(primer_numero.group()) if primer_numero else None

df_def['Precio'] = df_def['Precio'].apply(extraer_primer_valor)

df_def['Precio'] = df_def['Precio'].interpolate(method='linear')
#Transformamos la columna de precio a numerico
df_def['Precio'] = pd.to_numeric(df_def['Precio'])

#vemos el tipo de datos de cada columna
df_def.dtypes

Precio                   float64
Descripcion               object
Otras_caractericas        object
Caracteristicas           object
Eficiencia_energetica     object
dtype: object

Tras llamar al dataset y limpiar la columna del precio se va intentar construir un dataframe con columnas de m^2 número de habitaciones, baños etc.

In [2]:
import numpy as np
# Lista para almacenar los resultados
resultados = []

for i in range(len(df_def)):
    # Obtén la información sobre las habitaciones de la fila actual
    habitaciones_info = df_def["Caracteristicas"][i].get("habitaciones")
    
    # Verificar si habitaciones_info no es None
    if habitaciones_info:
        # Buscar el número que acompaña a la palabra "dormitorio"
        dormitorios_numero = re.search(r'(\d+) dormitorios', habitaciones_info)
        if dormitorios_numero:
            dormitorios = dormitorios_numero.group(1)
        else:
            dormitorios = None
    else:
        dormitorios = None
    
    # Agregar el resultado a la lista
    resultados.append(dormitorios)

habitaciones_series = pd.Series(resultados, dtype='float')

# Usar la interpolación lineal para llenar los valores faltantes
habitaciones_series = habitaciones_series.interpolate(method='linear', limit_direction='both')

# Redondear los resultados interpolados y convertir a enteros
resultados_imputados = habitaciones_series.round().astype('Int64').tolist()

# Agregar la lista imputada como una nueva columna al DataFrame
df_def['Habitaciones'] = resultados_imputados

# Mostrar el DataFrame con la nueva columna
print(df_def["Habitaciones"])


https://www.indomio.es/anuncios/94613177/                                          2
https://www.indomio.es/anuncios/95526627/                                          3
https://www.indomio.es/anuncios/95146207/                                          4
https://www.indomio.es/anuncios/94500757/                                          2
https://www.indomio.es/anuncios/95414773/                                          2
                                                                                  ..
https://www.pisos.com//comprar/piso-rascanya_torrefiel46025-42563861154_100500/    2
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-43429955909_521818/    2
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-44188952198_109300/    2
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-43417138120_100900/    2
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-44224797149_101800/    2
Name: Habitaciones, Length: 3217, dtype: int64


C:\Users\alexn\AppData\Local\Temp\ipykernel_19652\1926230695.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  habitaciones_info = df_def["Caracteristicas"][i].get("habitaciones")


In [3]:
resultados = []

for i in range(len(df_def)):
    # Obtén la información sobre los baños de la fila actual
    habitaciones_info = df_def["Caracteristicas"][i].get("habitaciones")
    
    # Verificar si habitaciones_info no es None
    if habitaciones_info:
        # Buscar el número que acompaña a la palabra "baño(s)"
        banos_numero = re.search(r'(\d+) baño[s]?', habitaciones_info)
        if banos_numero:
            banos = banos_numero.group(1)
        else:
            banos = None
    else:
        banos = None
    
    # Agregar el resultado a la lista
    resultados.append(banos)

baños_series = pd.Series(resultados, dtype='float')

# Usar la interpolación lineal para llenar los valores faltantes
baños_series = baños_series.interpolate(method='linear', limit_direction='both')

# Redondear los resultados interpolados y convertir a enteros
resultados_imputados = baños_series.round().astype('Int64').tolist()

# Agregar la lista imputada como una nueva columna al DataFrame
df_def['Baños'] = resultados_imputados

print(df_def["Baños"])



https://www.indomio.es/anuncios/94613177/                                          1
https://www.indomio.es/anuncios/95526627/                                          1
https://www.indomio.es/anuncios/95146207/                                          1
https://www.indomio.es/anuncios/94500757/                                          2
https://www.indomio.es/anuncios/95414773/                                          1
                                                                                  ..
https://www.pisos.com//comprar/piso-rascanya_torrefiel46025-42563861154_100500/    2
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-43429955909_521818/    2
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-44188952198_109300/    2
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-43417138120_100900/    2
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-44224797149_101800/    2
Name: Baños, Length: 3217, dtype: int64


C:\Users\alexn\AppData\Local\Temp\ipykernel_19652\3331557728.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  habitaciones_info = df_def["Caracteristicas"][i].get("habitaciones")


In [4]:
metros_cuadrados_lista = []
for i in range(len(df_def)):
    # Convertir la entrada de la columna "Descripcion" a cadena de texto
    texto = str(df_def["Descripcion"][i])
    
    # Buscar los metros cuadrados usando una expresión regular
    metros_cuadrados_match = re.search(r'(\d+(?:\.\d+)?)\s*(?:m²|m2|metros\s+cuadrados|metros)', texto)
    
    # Verificar si se encontró la información de los metros cuadrados
    if metros_cuadrados_match:
        metros_cuadrados = float(metros_cuadrados_match.group(1))  # Convertir a float en lugar de int
    else:
        metros_cuadrados = None
    
    # Agregar el resultado a la lista
    metros_cuadrados_lista.append(metros_cuadrados)



metros_cuadrados_series = pd.Series(metros_cuadrados_lista, dtype='float')

# Usar la interpolación lineal para llenar los valores faltantes
metros_cuadrados_series = metros_cuadrados_series.interpolate(method='linear', limit_direction='both')

# Redondear los resultados interpolados y convertir a enteros
resultados_imputados = metros_cuadrados_series.round().astype('Int64').tolist()

# Agregar la lista imputada como una nueva columna al DataFrame
df_def['Metros Cuadrados'] = resultados_imputados

print(df_def["Metros Cuadrados"])

C:\Users\alexn\AppData\Local\Temp\ipykernel_19652\1087256648.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  texto = str(df_def["Descripcion"][i])


https://www.indomio.es/anuncios/94613177/                                          120
https://www.indomio.es/anuncios/95526627/                                          120
https://www.indomio.es/anuncios/95146207/                                          120
https://www.indomio.es/anuncios/94500757/                                          115
https://www.indomio.es/anuncios/95414773/                                          110
                                                                                  ... 
https://www.pisos.com//comprar/piso-rascanya_torrefiel46025-42563861154_100500/     73
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-43429955909_521818/     62
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-44188952198_109300/     50
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-43417138120_100900/    101
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-44224797149_101800/    101
Name: Metros Cuadrados, Length: 3217, dtype

In [5]:
# Se crea una lista para almacenar los resultados
resultados = []
for i in range(len(df_def)):
    contrato = df_def["Caracteristicas"][i].get("contrato")
    
    if contrato:
        contrato = contrato.lower()
    else:
        contrato = None
    
    resultados.append(contrato)

df_def["Contrato"] = resultados

df_def["Contrato"] = df_def["Contrato"].fillna("venta")

print(df_def["Contrato"])

C:\Users\alexn\AppData\Local\Temp\ipykernel_19652\1888958447.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  contrato = df_def["Caracteristicas"][i].get("contrato")


https://www.indomio.es/anuncios/94613177/                                          alquiler
https://www.indomio.es/anuncios/95526627/                                          alquiler
https://www.indomio.es/anuncios/95146207/                                          alquiler
https://www.indomio.es/anuncios/94500757/                                          alquiler
https://www.indomio.es/anuncios/95414773/                                          alquiler
                                                                                     ...   
https://www.pisos.com//comprar/piso-rascanya_torrefiel46025-42563861154_100500/       venta
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-43429955909_521818/       venta
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-44188952198_109300/       venta
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-43417138120_100900/       venta
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-44224797149_101800/ 

In [6]:
#Se cambia el tipo de datos de las columnas habitaciones, baños a numerico y el de contrato a stirng

df_def["Habitaciones"] = pd.to_numeric(df_def["Habitaciones"])
df_def["Baños"] = pd.to_numeric(df_def["Baños"])
df_def["Contrato"] = df_def["Contrato"].astype(str)


df_def.dtypes

Precio                   float64
Descripcion               object
Otras_caractericas        object
Caracteristicas           object
Eficiencia_energetica     object
Habitaciones               int64
Baños                      int64
Metros Cuadrados           int64
Contrato                  object
dtype: object

In [7]:
import spacy
import re
import unicodedata
import string

# Cargar el modelo de spaCy para el idioma español
nlp = spacy.load("es_core_news_sm")

def remove_accents(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text

def remove_special_characters(text):
    pat = '[{}]'.format(re.escape(string.punctuation))
    return re.sub(pat, '', text)

def preprocess_text(text):
    if text is None:
        return ''
    # Convertir texto a minúsculas
    text = text.lower()
    # Eliminar acentos
    text = remove_accents(text)
    # Eliminar caracteres especiales y números
    text = remove_special_characters(text)
    # Tokenización y lematización con spaCy
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc]
    # Eliminar stopwords (opcional)
    tokens = [word for word in tokens if not nlp.vocab[word].is_stop]
    return ' '.join(tokens)

# Aplicar la función de preprocesado a la columna 'Descripcion'
df_def['Descripcion'] = df_def['Descripcion'].apply(preprocess_text)

print(df_def['Descripcion'])


https://www.indomio.es/anuncios/94613177/                                          disponible alquiler mensual anual 2 dormitorio...
https://www.indomio.es/anuncios/95526627/                                          alquilar piso amueblado 3 habitación carrer de...
https://www.indomio.es/anuncios/95146207/                                          piso estudiantes entrar 1ro febrero 30 junio p...
https://www.indomio.es/anuncios/94500757/                                          disponible alquiler mensual anual 2 dormitorio...
https://www.indomio.es/anuncios/95414773/                                          precioso piso 2 dormitorio ubicacion ideal min...
                                                                                                         ...                        
https://www.pisos.com//comprar/piso-rascanya_torrefiel46025-42563861154_100500/    residencial moncado 106 \r  disponer piso atic...
https://www.pisos.com//comprar/piso-rascanya_torrefiel46019-434299559

In [8]:
# Seleccionar las columnas relevantes
columns = ['Descripcion', 'Precio', 'Habitaciones', 'Baños', 'Metros Cuadrados', 'Contrato']
df_filtered = df_def[columns]
df_filtered.to_pickle("pisos_filtrado.pkl")
min_bedrooms = 2
max_price = 1500
min_bathrooms = 1
min_sqft = 120
contrato = "alquiler"

# List to keep track of applied filters
applied_filters = []

# Filtrar el DataFrame por habitaciones
if min_bedrooms is not None and len(df_filtered[df_filtered['Habitaciones'] >= min_bedrooms]) >= 5:
    df_filtered = df_filtered[df_filtered['Habitaciones'] >= min_bedrooms]
    applied_filters.append('Habitaciones')

# Filtrar el DataFrame por precio
if max_price is not None and len(df_filtered[df_filtered['Precio'] <= max_price]) >= 5:
    df_filtered = df_filtered[df_filtered['Precio'] <= max_price]
    applied_filters.append('Precio')

# Filtrar el DataFrame por baño
if min_bathrooms is not None and len(df_filtered[df_filtered['Baños'] >= min_bathrooms]) >= 5:
    df_filtered = df_filtered[df_filtered['Baños'] >= min_bathrooms]
    applied_filters.append('Baños')

# Filtrar el DataFrame por metros cuadrados
if min_sqft is not None and len(df_filtered[df_filtered['Metros Cuadrados'] >= min_sqft]) >= 5:
    df_filtered = df_filtered[df_filtered['Metros Cuadrados'] >= min_sqft]
    applied_filters.append('Metros cuadrados')

# Filtrar el DataFrame por contrato
if contrato is not None and len(df_filtered[df_filtered['Contrato'] == contrato]) >= 5:
    df_filtered = df_filtered[df_filtered['Contrato'] == contrato]
    applied_filters.append('Contrato')

print("Applied filters:", applied_filters)

Applied filters: ['Habitaciones', 'Precio', 'Baños', 'Metros cuadrados', 'Contrato']


In [9]:
user_description = "Un piso de alquiler en Campanar por menos de 1500 euros al mes, con tres habitaciones, dos baños y terraza. Uno de los baños tiene bañera y el otro ducha. El salón debe ser espacioso y luminoso y la cocina debe ser moderna y estar amueblada"
user_desc_processed = [preprocess_text(user_description)]
print(user_desc_processed)

['piso alquiler campanar 1500 euros mes habitación bano terrazar bano banera ducha salon deber espacioso luminoso cocina deber moderno amueblado']


### BOW

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

cv = CountVectorizer()
user_desc_bow = cv.fit_transform(user_desc_processed)
desc_bow = cv.transform(df_filtered['Descripcion'])
vocab = cv.get_feature_names_out()

similarity_bow = cosine_similarity(user_desc_bow, desc_bow)

print(vocab)

['1500' 'alquiler' 'amueblado' 'banera' 'bano' 'campanar' 'cocina' 'deber'
 'ducha' 'espacioso' 'euros' 'habitación' 'luminoso' 'mes' 'moderno'
 'piso' 'salon' 'terrazar']


In [11]:
import numpy as np

sorted_indices = np.argsort(similarity_bow[0,:])

filtered_df_bow = df_filtered.iloc[sorted_indices]

print(filtered_df_bow)

                                                                                 Descripcion  \
https://www.indomio.es/anuncios/94845913/  alquilar departamento amoblado 2 recamara av p...   
https://www.indomio.es/anuncios/94374769/  loft completamente equipado compuesto cama dob...   
https://www.indomio.es/anuncios/95139179/           alquilar monoambiente amueblado c trench   
https://www.indomio.es/anuncios/95583061/  alquilar estudio amueblado carrer colon propie...   
https://www.indomio.es/anuncios/94711655/  alquilar estudio amueblado c francesc cubells ...   
https://www.indomio.es/anuncios/95315055/  alquilar piso 1 dormitorio amueblado carrer ca...   
https://www.indomio.es/anuncios/94960209/  habitacion alquilerencuentra hogar ideal piso ...   
https://www.indomio.es/anuncios/95210501/    punto dulce   ohtar adorable   gustara   dep...   
https://www.indomio.es/anuncios/95048725/  alquilar piso 1 dormitorio amueblado carrer pu...   
https://www.indomio.es/anuncios/95033983

### TFIDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer()
user_desc_tfidf = tv.fit_transform(user_desc_processed)
desc_tfidf = tv.transform(df_filtered['Descripcion'])
vocab = tv.get_feature_names_out()

similarity_tfidf = cosine_similarity(user_desc_tfidf, desc_tfidf)

print(vocab)

['1500' 'alquiler' 'amueblado' 'banera' 'bano' 'campanar' 'cocina' 'deber'
 'ducha' 'espacioso' 'euros' 'habitación' 'luminoso' 'mes' 'moderno'
 'piso' 'salon' 'terrazar']


In [13]:
import numpy as np

sorted_indices = np.argsort(similarity_tfidf[0,:])

filtered_df_tfidf = df_filtered.iloc[sorted_indices]

print(filtered_df_tfidf)

                                                                                 Descripcion  \
https://www.indomio.es/anuncios/94845913/  alquilar departamento amoblado 2 recamara av p...   
https://www.indomio.es/anuncios/94374769/  loft completamente equipado compuesto cama dob...   
https://www.indomio.es/anuncios/95139179/           alquilar monoambiente amueblado c trench   
https://www.indomio.es/anuncios/95583061/  alquilar estudio amueblado carrer colon propie...   
https://www.indomio.es/anuncios/94711655/  alquilar estudio amueblado c francesc cubells ...   
https://www.indomio.es/anuncios/95315055/  alquilar piso 1 dormitorio amueblado carrer ca...   
https://www.indomio.es/anuncios/94960209/  habitacion alquilerencuentra hogar ideal piso ...   
https://www.indomio.es/anuncios/95048725/  alquilar piso 1 dormitorio amueblado carrer pu...   
https://www.indomio.es/anuncios/95210501/    punto dulce   ohtar adorable   gustara   dep...   
https://www.indomio.es/anuncios/95033983